In [ ]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.pardir))
from argparse import ArgumentParser
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = (20, 20)
import seaborn as sns
plt.style.use('ggplot')
from PIL import Image
import cv2
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import plotly.graph_objects as go

import json 
from tqdm import tqdm


import shapely
# sys.path.append(cellhier_path)

# args = get_args() 
hnes = sorted(glob.glob(f"hne/*.tif"))
masks = sorted(glob.glob(f"masks/*.tiff"))

def get_cell_type_count(mask, mask_cells):
    
    # height, width = mask.shape 
    # mask = cv2.resize(mask, (width * 2, height * 2), interpolation=cv2.INTER_AREA)
    
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    cell_type_count_in_ftu = {}
    for index, ftu_contour in enumerate(tqdm(contours)):
        try:
            ftu_polygon = Polygon(np.squeeze(ftu_contour))
        except ValueError as e:
            print(e)
            continue
        cell_type_count_in_ftu[index] = {
            'ftu': ftu_contour.tolist(), 
            'cells': [],
            'cell_type_count': {}
        }
        for x, y, cell_type in zip(mask_cells['x'], mask_cells['y'], mask_cells['Cell Type']):
            point = Point(x, y)
            if ftu_polygon.contains(point):
                cell_type_count_in_ftu[index]['cells'].append([x, y, cell_type])
                cell_type_count_in_ftu[index]['cell_type_count'].setdefault(cell_type, 0)
                cell_type_count_in_ftu[index]['cell_type_count'][cell_type] += 1
                                    
    return cell_type_count_in_ftu 

def plot_cell_type_count(hne_name, mask, mask_cells):
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cell_type_count_in_ftu = get_cell_type_count(mask, mask_cells)

    fig=go.Figure()
    fig.add_scatter(x=np.arange(0, 7), y=np.random.randint(2, 13, 7), 
                    line_color="red", line_width=2)
    fig.update_layout(template="plotly_white", width=400, height=400,
                    xaxis_showgrid=False, yaxis_showgrid=False)
    fig.add_layout_image(
            source=hne_name,
            xref="paper",
            yref="paper",
            x=0,
            y=1,
            xanchor="left",
            yanchor="top",
            layer="below",
            sizing="stretch",
            sizex=1.0,
            sizey=1.0
        )
    
    fig.show()

# load cells
X = "X"
Y = "Y"
cluster_col = "Cell Type"
cells = pd.read_csv("/Users/abhiroop/Developer/cns/CODEX_HuBMAP_alldata_Dryad.csv")
cells = pd.concat([cells,pd.get_dummies(cells[cluster_col])], 1)

for hne_name, mask_name in zip(hnes, masks):
    hne = np.array(Image.open(hne_name))
    mask = np.array(Image.open(mask_name))
    array, region, _, _ = mask_name.split('/')[1].split('_')
    region = int(region[-1])
    # corresponding cells 
    mask_cells = cells[cells['array'] == array]
    mask_cells = mask_cells[mask_cells['region'] == region]

    height, width = mask.shape 
    mask = cv2.resize(mask, (width * 2, height * 2), interpolation=cv2.INTER_AREA)

    # plot_cell_type_count(hne_name, mask, mask_cells)
    
    cell_type_count = get_cell_type_count(mask, mask_cells)
    break
    # with open("cell_type_ftu_4.json", "w") as outfile:
    #     json.dump(cell_type_count, outfile)

    # # computed as sum(cell_type_counts in all ftu) / no_of_ftus
    # mean_cell_type_counts = {}
    # ftu_count = 0
    # for index in cell_type_count.keys():
    #     ftu_count += 1
    #     for cell_type in cell_type_count[index]['cell_type_count'].keys():
    #         mean_cell_type_counts.setdefault(cell_type, 0)
    #         mean_cell_type_counts[cell_type] += cell_type_count[index]['cell_type_count'][cell_type]
            
    # for cell_type in mean_cell_type_counts.keys():
    #     mean_cell_type_counts[cell_type] = round(mean_cell_type_counts[cell_type] / ftu_count)


    # cellxBP = {
    #     "@context": "https://hubmapconsortium.github.io/ccf-ontology/ccf-context.jsonld",
    #     "@graph": [
    #         {
    #         "@id": None,
    #         "@type": "CellTypeSummary",
    #         "dataset": None,
    #         "summary": [
                
    #         ]
    #         }
    #     ]
        # }






In [ ]:
cell_type_count[0]

In [ ]:
np.array(cell_type_count[0]['ftu'])[:, :, 0]

In [ ]:
np.array(cell_type_count[0]['ftu'])[:, :, 1]

In [ ]:
hne_name

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
# Add image
img_width = 1600
img_height = 900
scale_factor = 0.5
fig.add_layout_image(
        x=0,
        sizex=img_width,
        y=0,
        sizey=img_height,
        xref="x",
        yref="y",
        opacity=1.0,
        layer="below",
        source=Image.fromarray(hne)
)
fig.update_xaxes(showgrid=False, range=(0, img_width))
fig.update_yaxes(showgrid=False, scaleanchor='x', range=(img_height, 0))
# Line shape added programatically
fig.add_shape(
    type='line', xref='x', yref='y',
    x0=650, x1=1080, y0=380, y1=180, line_color='cyan'
)
# Set dragmode and newshape properties; add modebar buttons
fig.update_layout(
    dragmode='drawrect',
    newshape=dict(line_color='cyan'),
    title_text='Drag to add annotations - use modebar to change drawing tool'
)
fig.show(config={'modeBarButtonsToAdd':['drawline',
                                        'drawopenpath',
                                        'drawclosedpath',
                                        'drawcircle',
                                        'drawrect',
                                        'eraseshape'
                                       ]})

In [ ]:
import glob 
import shutil 

hne_dir = glob.glob("/Users/abhiroop/Developer/cns/hra-interactive-ftu-computation/HandE_annot_High_res/HandE_B009toB1012/B012B/*.tif")
out_dir = '/Users/abhiroop/Developer/cns/hra-interactive-ftu-computation/HandE_annot_High_res/HandE_B009toB1012'

for hne in hne_dir:
    shutil.copy(hne, f'{out_dir}/B012B_{hne.split("/")[-1]}')


In [ ]:
fig = go.Figure()
fig.add_scatter(x=np.arange(0, 7), y=np.random.randint(2, 13, 7), mode='markers')
fig.update_layout(template="plotly_white", width=400, height=400,
                xaxis_showgrid=False, yaxis_showgrid=False)
fig.add_layout_image(
        source=hne_name,
    )


In [ ]:
import plotly.express as px

fig = px.imshow(hne)
fig.add_trace(go.Contour(z=mask, showscale=False,
                         contours=dict(start=0, end=70, size=70, coloring='lines'),
                         line_width=2))